In [92]:
import torch
import requests
from tqdm import tqdm
from torch import nn
from torch.nn import Embedding, Linear, Bilinear, BatchNorm1d, ReLU, Dropout, MarginRankingLoss
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 

In [84]:
class PosNegData:

    def __init__(self, pos_data, neg_data, weight):
        self.pos = pos_data
        self.neg = neg_data
        self.weight = weight


class Data:

    def __init__(self, user_id, item_id, metadata):
        self.user_id = torch.tensor(user_id)
        self.item_id = torch.tensor(item_id)
        self.metadata = torch.tensor(metadata)


class DataGenerator(Dataset):

    def __init__(self, state_history, reward_history, action_history):
        self.state_history = state_history
        self.reward_history = reward_history
        self.action_history = action_history
        self.data = []
        self._init_pos_neg()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def _init_pos_neg(self):
        for i, r in enumerate(self.reward_history):
            if r > 0:
                user_id = self.state_history[i][0][0]
                action = self.action_history[i]
                pos_data = Data(user_id=user_id, item_id=self.state_history[i][action][1],
                                metadata=self.state_history[i][action][2:])
                for j, state in enumerate(self.state_history[i]):
                    item_id = state[1]
                    metadata = state[2:]
                    data = Data(user_id=user_id, item_id=item_id, metadata=metadata)
                    if j != action:
                        self.data.append(PosNegData(pos_data, data, 1))

    def add_data(self, state, action, reward):
        if reward > 0:
            user_id = state[0][0]
            pos_data = Data(user_id=user_id, item_id=state[action][1], metadata=state[action][2:])
            for j, my_state in enumerate(state):
                item_id = my_state[1]
                metadata = my_state[2:]
                data = Data(user_id=user_id, item_id=item_id, metadata=metadata)
                if j != action:
                    self.data.append(PosNegData(pos_data, data, 1))


def collate_data_pos_neg(list_of_data):
    raw_data = [data for data in list_of_data]
    user_id_pos = torch.stack([data.pos.user_id for data in list_of_data])
    item_id_pos = torch.stack([data.pos.item_id for data in list_of_data])
    metadata_pos = torch.stack([data.pos.metadata for data in list_of_data])
    user_id_neg = torch.stack([data.neg.user_id for data in list_of_data])
    item_id_neg = torch.stack([data.neg.item_id for data in list_of_data])
    metadata_neg = torch.stack([data.neg.metadata for data in list_of_data])
    return {'user_id_pos': user_id_pos, 'item_id_pos': item_id_pos, 'metadata_pos': metadata_pos, 'raw_data': raw_data,
            'user_id_neg': user_id_neg, 'item_id_neg': item_id_neg, 'metadata_neg': metadata_neg}


def collate_data(list_of_data):
    user_id = torch.stack([data.user_id for data in list_of_data])
    item_id = torch.stack([data.item_id for data in list_of_data])
    metadata = torch.stack([data.metadata for data in list_of_data])
    return {'user_id': user_id, 'item_id': item_id, 'metadata': metadata}


In [85]:
class Interface:

    def __init__(self, args):
        self.base_url = 'http://{}'.format(args.ip_address_env_2)
        self.user_id = args.user_id
        self.url_reset = '{}/reset'.format(self.base_url)
        self.url_predict = '{}/predict'.format(self.base_url)

        r = requests.get(url=self.url_reset, params={'user_id': self.user_id})
        data = r.json()
        self.state_history = data['state_history']
        self.rewards_history = data['rewards_history']
        self.action_history = data['action_history']

        self.nb_items = data['nb_items']
        self.nb_users = data['nb_users']
        self.nb_variables = len(self.state_history[0][0]) - 2

        self.next_state = data['next_state']

    def reset(self):
        r = requests.get(url=self.url_reset, params={'user_id': self.user_id})
        data = r.json()

        self.state_history = data['state_history']
        self.rewards_history = data['rewards_history']
        self.action_history = data['action_history']

        self.nb_items = data['nb_items']
        self.nb_users = data['nb_users']

        self.next_state = data['next_state']

    def predict(self, recommended_item):
        r = requests.get(url=self.url_predict, params={'user_id': self.user_id, 'recommended_item': recommended_item})
        data = r.json()

        self.state_history.append(data['state'])
        self.rewards_history.append(data['reward'])
        self.action_history.append(recommended_item)

        self.next_state = data['state']
        return data['state'], data['reward']


In [95]:
class SiameseNetwork(nn.Module):

    def __init__(self, interface):
        super(SiameseNetwork, self).__init__()

        user_embedding_dim = 10
        item_embedding_dim = 10
        user_meta_dim = 30
        item_meta_dim = 30
        meta_meta_dim = 90
        dense_1_dim = 128
        dense_2_dim = 64
        out_dim = 1

        self.embedding_user = Embedding(num_embeddings=interface.nb_users, embedding_dim=user_embedding_dim)
        self.embedding_item = Embedding(num_embeddings=interface.nb_items, embedding_dim=item_embedding_dim)
        self.concat_user_meta = Bilinear(in1_features=user_embedding_dim, in2_features=interface.nb_variables, out_features=user_meta_dim)
        self.concat_item_meta = Bilinear(in1_features=item_embedding_dim, in2_features=interface.nb_variables, out_features=item_meta_dim)
        self.concat_meta_meta = Bilinear(in1_features=user_meta_dim, in2_features=item_meta_dim, out_features=meta_meta_dim)
        self.batch_norm_0 = BatchNorm1d(num_features=meta_meta_dim)
        self.dropout_0 = Dropout(0.5)
        self.dense_1 = Linear(in_features=meta_meta_dim, out_features=dense_1_dim)
        self.relu_1 = ReLU()
        self.dropout_1 = Dropout(0.5)
        self.batch_norm_1 = BatchNorm1d(num_features=dense_1_dim)
        self.dense_2 = Linear(in_features=dense_1_dim, out_features=dense_2_dim)
        self.relu_2 = ReLU()
        self.dropout_2 = Dropout(0.5)
        self.batch_norm_2 = BatchNorm1d(num_features=dense_2_dim)
        self.dense_3 = Linear(in_features=dense_2_dim, out_features=out_dim)

    def forward(self, user_id, item_id, metadata):
        user_embedded = self.embedding_user(user_id).squeeze(dim=1)
        item_embedded = self.embedding_item(item_id).squeeze(dim=1)
        user_and_meta = self.concat_user_meta(user_embedded, metadata)
        item_and_meta = self.concat_item_meta(item_embedded, metadata)
        meta_and_meta = self.concat_meta_meta(user_and_meta, item_and_meta)
        output = self.batch_norm_0(meta_and_meta)
        # output = self.dropout_0(output)
        output = self.dense_1(output)
        output = self.relu_1(output)
        output = self.batch_norm_1(output)
        # output = self.dropout_1(output)
        output = self.dense_2(output)
        output = self.relu_2(output)
        output = self.batch_norm_2(output)
        # output = self.dropout_2(output)
        output = self.dense_3(output)
        return output


In [98]:
class Trainer:

    def __init__(self, interface, learning_rate=1e-3, validation_split=0.2, batch_size=64, margin=10, min_weight=1,
                 num_samples=30):
        self.interface = interface
        self.network = SiameseNetwork(interface)
        self.dataset = DataGenerator(interface.state_history, interface.rewards_history, interface.action_history)
        self.batch_size = batch_size
        self.validation_split = validation_split
        self.min_weight = min_weight
        self.num_samples = num_samples

        self.loss = MarginRankingLoss(margin=margin, reduction='none')

        self.optimizer = Adam(self.network.parameters(), lr=learning_rate)
        self.lr_scheduler = ReduceLROnPlateau(self.optimizer, factor=0.3, patience=3, threshold=1e-3, verbose=True)

    def reset(self):
        self.train()

    def train(self, n=600):
        for _ in range(n):
            weights = [data.weight for data in self.dataset]
            sampler = WeightedRandomSampler(weights=weights, num_samples=self.num_samples, replacement=True)
            data_loader = DataLoader(self.dataset, batch_size=self.batch_size, sampler=sampler,
                                     collate_fn=collate_data_pos_neg, drop_last=True)
            self.network.train()
            cumloss = 0
            for inputs in data_loader:
                self.optimizer.zero_grad()
                output_pos = self.network(inputs['user_id_pos'], inputs['item_id_pos'], inputs['metadata_pos'])
                output_neg = self.network(inputs['user_id_neg'], inputs['item_id_neg'], inputs['metadata_neg'])
                loss = self.loss(output_pos, output_neg, torch.ones(output_pos.shape))
                for j, data in enumerate(inputs['raw_data']):
                    data.weight = loss[j][0].item()
                cumloss += loss.sum().item()
                loss = loss.mean()
                loss.backward()
                self.optimizer.step()

    def online(self):
        self.network.eval()
        l = []
        my_state = self.interface.next_state
        for m in self.interface.next_state:
            data = Data(m[0], m[1], m[2:])
            l.append(data)
        input = collate_data(l)
        output = self.network(input['user_id'], input['item_id'], input['metadata']).squeeze()
        recommended_item = output.argmax().item()
        state, reward = self.interface.predict(recommended_item)
        self.dataset.add_data(my_state, recommended_item, reward)
        self.train(n=3)
        return reward


In [99]:
class Argument:
    pass


list_reward = []

for _ in range(100):
    args = Argument
    args.user_id = 'R3EIFXNYY6XMBXBR01BK'
    args.ip_address_env_0 = '52.47.62.31'
    args.ip_address_env_1 = '35.180.254.42'
    args.ip_address_env_2 = '35.180.178.243'

    interface = Interface(args)
    trainer = Trainer(interface)
    interface.reset()
    trainer.reset()

    reward = 0
    nb_iter = 100

    for k in range(nb_iter):
        if (k % 10 == 9):
            print("\rIteration {}\tTotal Reward {:5f}\tMean Reward {:5f}   ".format(
                k, reward, reward / k), end="")
        # makes a prediction, returns the reward and update the network
        reward += trainer.online()

    print("Total reward : " + str(reward))
    print("Mean reward : " + str(reward / nb_iter))
    list_reward.append(reward)
plt.hist(list_reward, bins=5)
print('mean: {}'.format(np.mean(list_reward)))
print('variance: {}'.format(np.std(list_reward)))
plt.show()

Iteration 9	Total Reward 3286.871150	Mean Reward 365.207906   

Iteration 19	Total Reward 5641.949094	Mean Reward 296.944689   

Iteration 29	Total Reward 7520.656052	Mean Reward 259.332967   

Iteration 39	Total Reward 8965.423747	Mean Reward 229.882660   

Iteration 49	Total Reward 10545.903939	Mean Reward 215.222529   

...

Iteration 19	Total Reward 7443.682304	Mean Reward 391.772753   

Iteration 29	Total Reward 8311.535274	Mean Reward 286.604665   

Iteration 39	Total Reward 12260.218748	Mean Reward 314.364583   

Iteration 49	Total Reward 13575.174400	Mean Reward 277.044376   

Iteration 59	Total Reward 15310.880340	Mean Reward 259.506446   